In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
import time


In [3]:
dataset=pd.read_csv('train.csv', parse_dates=['Dates'])

testset=pd.read_csv('test.csv',parse_dates=['Dates'])

testset['Year'] = testset['Dates'].map(lambda x: x.year)
testset['Month'] = testset['Dates'].map(lambda x: x.month)
testset['Hour'] = testset['Dates'].map(lambda x: x.hour)

dataset['Year'] = dataset['Dates'].map(lambda x: x.year)
dataset['Month'] = dataset['Dates'].map(lambda x: x.month)

dataset['Hour'] = dataset['Dates'].map(lambda x: x.hour)

In [4]:
dataset['Y']=dataset['Y'].apply(lambda x : x if 37.82 > x else 37.82)
dataset['X']=dataset['X'].apply(lambda x : x if -122.3 > x else 122.3)

In [5]:
dataset["rot45_X"]=0.707 * dataset["Y"] + 0.707 * dataset["X"]
dataset["rot45_Y"]=0.707 * dataset["Y"] - 0.707 * dataset["X"]


In [6]:
dataset

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Year,Month,Hour,rot45_X,rot45_Y
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,2013,6,17,-59.852546,113.248388
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2004,2,2,-59.833459,113.256175
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,2007,11,0,-59.831985,113.246430
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,2007,12,18,-59.864910,113.215357
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,2012,9,17,-59.826321,113.254686
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,2014,5,6,-59.848713,113.250911
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,2006,8,23,-59.822246,113.257930
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,2012,11,12,-59.864054,113.224917
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,2007,2,23,-59.863773,113.218185
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,2006,2,7,-59.835866,113.262776


In [7]:
#dataset=dataset.drop(['X'],axis=1)


In [8]:
#dataset=dataset.drop(['Y'],axis=1)

In [9]:
dataset["radial45"]=np.sqrt(np.power(dataset['rot45_X'],2) + np.power(dataset['rot45_Y'],2))

In [10]:
dataset

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Year,Month,Hour,rot45_X,rot45_Y,radial45
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,2013,6,17,-59.852546,113.248388,128.091860
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2004,2,2,-59.833459,113.256175,128.089828
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,2007,11,0,-59.831985,113.246430,128.080522
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,2007,12,18,-59.864910,113.215357,128.068437
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,2012,9,17,-59.826321,113.254686,128.085177
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,2014,5,6,-59.848713,113.250911,128.092300
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,2006,8,23,-59.822246,113.257930,128.086142
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,2012,11,12,-59.864054,113.224917,128.076488
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,2007,2,23,-59.863773,113.218185,128.070405
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,2006,2,7,-59.835866,113.262776,128.096789


In [11]:
dataset["rot60_X"]=(0.5) * dataset["Y"] + (1.732/2) * dataset["X"]
dataset["rot60_Y"]=0.5 * dataset["Y"] - (1.732/2) * dataset["X"]


In [12]:
dataset["radial60"]=np.sqrt(np.power(dataset['rot60_X'],2) + np.power(dataset['rot60_Y'],2))

In [13]:
dataset["rot30_X"]=(1.732/2) * dataset["Y"] + 0.5 * dataset["X"]
dataset["rot30_Y"]=(1.732/2) * dataset["Y"] - 0.5 * dataset["X"]


In [14]:
dataset["radial30"]=np.sqrt(np.power(dataset['rot30_X'],2) + np.power(dataset['rot30_Y'],2))

In [15]:
dataset["radial"]=np.sqrt(np.power(dataset['X'],2) + np.power(dataset['Y'],2))

In [17]:
dataset['Hour']

0         17
1          2
2          0
3         18
4         17
5          6
6         23
7         12
8         23
9          7
10        17
11         3
12        20
13        15
14        13
15         4
16        11
17        22
18         0
19         2
20         7
21         9
22         2
23        10
24        10
25        12
26        12
27        14
28        15
29         8
          ..
867843    17
867844    22
867845    18
867846    17
867847    16
867848    19
867849    16
867850     9
867851    22
867852    18
867853    11
867854    12
867855    10
867856    10
867857     8
867858    18
867859     7
867860     0
867861     8
867862    14
867863    15
867864    19
867865    14
867866    12
867867    10
867868    17
867869    22
867870    22
867871     5
867872    12
Name: Hour, Length: 867873, dtype: int64

In [41]:
dataset.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Hour',
       'rot45_X', 'rot45_Y', 'radial45', 'rot60_X', 'rot60_Y', 'radial60',
       'rot30_X', 'rot30_Y', 'radial30', 'radial'],
      dtype='object')

In [42]:
dataset['Month']

0          6
1          2
2         11
3         12
4          9
5          5
6          8
7         11
8          2
9          2
10         7
11         2
12         1
13         1
14         9
15         9
16        10
17         8
18        10
19         9
20        11
21         6
22         8
23         3
24         2
25         7
26         7
27        11
28         9
29        12
          ..
867843     7
867844    11
867845     7
867846     3
867847    12
867848     3
867849     9
867850     5
867851    11
867852     3
867853    12
867854     3
867855     7
867856    10
867857    10
867858     7
867859    10
867860     1
867861     3
867862     1
867863     7
867864     8
867865    12
867866    11
867867     3
867868    11
867869     4
867870     8
867871    11
867872     9
Name: Month, Length: 867873, dtype: int64

In [53]:
dataset['Month']=dataset['Month'].apply(lambda x : 'MonthLow' if x== 12 else ('MonthMed' if x in (2,6,7,8,9,11) else 'MonthHigh'))
dataset['HourofDay']=dataset['Hour'].apply(lambda x : 'EARLY_Morning' if (x >= 1) & (x <=5) else ('MORNING' if x > 5 & x <12 else ('DAY' if x>11 & x<19 else 'Night')))

In [54]:
dataset

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,...,rot45_Y,radial45,rot60_X,rot60_Y,radial60,rot30_X,rot30_Y,radial30,radial,HourofDay
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,...,113.248388,128.091860,-87.134009,124.896273,152.287276,-28.507545,93.911786,98.143282,128.111207,MORNING
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,...,113.256175,128.089828,-87.117585,124.898855,152.279998,-28.487089,93.924250,98.149269,128.109174,EARLY_Morning
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,...,113.246430,128.080522,-87.113638,124.889059,152.269705,-28.488188,93.915217,98.140944,128.099867,Night
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,...,113.215357,128.068437,-87.137403,124.867564,152.265674,-28.528038,93.876677,98.115643,128.087780,MORNING
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,...,113.254686,128.085177,-87.110304,124.895569,152.273137,-28.480580,93.924659,98.147771,128.104523,MORNING
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,...,113.250911,128.092300,-87.130958,124.897718,152.286716,-28.503188,93.915217,98.145299,128.111646,MORNING
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,...,113.257930,128.086142,-87.107207,124.897648,152.273071,-28.475803,93.928848,98.150394,128.105488,MORNING
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,...,113.224917,128.076488,-87.139051,124.876577,152.274009,-28.524738,93.886133,98.123731,128.095832,MORNING
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,...,113.218185,128.070405,-87.137037,124.870001,152.267463,-28.526208,93.879702,98.118006,128.089748,MORNING
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,...,113.262776,128.096789,-87.121619,124.905855,152.288047,-28.487707,93.930004,98.154954,128.116136,MORNING


In [61]:
XX=pd.DataFrame(dataset)

In [62]:
dataset

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,...,rot45_Y,radial45,rot60_X,rot60_Y,radial60,rot30_X,rot30_Y,radial30,radial,HourofDay
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,...,113.248388,128.091860,-87.134009,124.896273,152.287276,-28.507545,93.911786,98.143282,128.111207,MORNING
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,...,113.256175,128.089828,-87.117585,124.898855,152.279998,-28.487089,93.924250,98.149269,128.109174,EARLY_Morning
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,...,113.246430,128.080522,-87.113638,124.889059,152.269705,-28.488188,93.915217,98.140944,128.099867,Night
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,...,113.215357,128.068437,-87.137403,124.867564,152.265674,-28.528038,93.876677,98.115643,128.087780,MORNING
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,...,113.254686,128.085177,-87.110304,124.895569,152.273137,-28.480580,93.924659,98.147771,128.104523,MORNING
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,...,113.250911,128.092300,-87.130958,124.897718,152.286716,-28.503188,93.915217,98.145299,128.111646,MORNING
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,...,113.257930,128.086142,-87.107207,124.897648,152.273071,-28.475803,93.928848,98.150394,128.105488,MORNING
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,...,113.224917,128.076488,-87.139051,124.876577,152.274009,-28.524738,93.886133,98.123731,128.095832,MORNING
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,...,113.218185,128.070405,-87.137037,124.870001,152.267463,-28.526208,93.879702,98.118006,128.089748,MORNING
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,...,113.262776,128.096789,-87.121619,124.905855,152.288047,-28.487707,93.930004,98.154954,128.116136,MORNING


In [56]:
dataset['Month']

0          MonthMed
1          MonthMed
2          MonthMed
3          MonthLow
4          MonthMed
5         MonthHigh
6          MonthMed
7          MonthMed
8          MonthMed
9          MonthMed
10         MonthMed
11         MonthMed
12        MonthHigh
13        MonthHigh
14         MonthMed
15         MonthMed
16        MonthHigh
17         MonthMed
18        MonthHigh
19         MonthMed
20         MonthMed
21         MonthMed
22         MonthMed
23        MonthHigh
24         MonthMed
25         MonthMed
26         MonthMed
27         MonthMed
28         MonthMed
29         MonthLow
            ...    
867843     MonthMed
867844     MonthMed
867845     MonthMed
867846    MonthHigh
867847     MonthLow
867848    MonthHigh
867849     MonthMed
867850    MonthHigh
867851     MonthMed
867852    MonthHigh
867853     MonthLow
867854    MonthHigh
867855     MonthMed
867856    MonthHigh
867857    MonthHigh
867858     MonthMed
867859    MonthHigh
867860    MonthHigh
867861    MonthHigh


In [57]:
X=dataset[['Id','Year','Month','Hour','Descript','Address' ,'Category', 'DayOfWeek', 'PdDistrict','Resolution' ,'rot45_X', 'rot45_Y']]

In [58]:
X


,Id,Year,Month,Hour,Descript,Address,Category,DayOfWeek,PdDistrict,Resolution,rot45_X,rot45_Y
0,141546,2013,MonthMed,17,"FORCIBLE RAPE, BODILY FORCE",2100 Block of MISSION ST,SEX OFFENSES FORCIBLE,Friday,MISSION,NONE,-59.852546,113.248388
1,794152,2004,MonthMed,2,CONSUMING ALCOHOL IN PUBLIC VIEW,1000 Block of MARKET ST,LIQUOR LAWS,Thursday,SOUTHERN,"ARREST, BOOKED",-59.833459,113.256175
2,531205,2007,MonthMed,0,"CREDIT CARD, THEFT BY USE OF",800 Block of BRYANT ST,FRAUD,Wednesday,SOUTHERN,NONE,-59.831985,113.246430
3,523137,2007,MonthLow,18,ROBBERY OF A CHAIN STORE WITH A GUN,2400 Block of SAN BRUNO AV,ROBBERY,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,-59.864910,113.215357
4,200968,2012,MonthMed,17,PROBATION VIOLATION,4TH ST / STEVENSON ST,OTHER OFFENSES,Sunday,SOUTHERN,"ARREST, BOOKED",-59.826321,113.254686
5,75749,2014,MonthHigh,6,ENROUTE TO DEPARTMENT OF CORRECTIONS,15TH ST / NATOMA ST,WARRANTS,Wednesday,MISSION,"ARREST, BOOKED",-59.848713,113.250911
6,617521,2006,MonthMed,23,"BURGLARY, UNLAWFUL ENTRY",0 Block of MARK LN,BURGLARY,Wednesday,CENTRAL,"ARREST, BOOKED",-59.822246,113.257930
7,186953,2012,MonthMed,12,SUSPICIOUS OCCURRENCE,100 Block of PUTNAM ST,SUSPICIOUS OCC,Friday,INGLESIDE,NONE,-59.864054,113.224917
8,584559,2007,MonthMed,23,"DRIVERS LICENSE, SUSPENDED OR REVOKED",HALE ST / SAN BRUNO AV,OTHER OFFENSES,Monday,BAYVIEW,"ARREST, CITED",-59.863773,113.218185
9,653619,2006,MonthMed,7,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,ELLIS ST / LARKIN ST,DRUG/NARCOTIC,Tuesday,NORTHERN,"ARREST, BOOKED",-59.835866,113.262776


In [59]:
dataset['Hour']

0         17
1          2
2          0
3         18
4         17
5          6
6         23
7         12
8         23
9          7
10        17
11         3
12        20
13        15
14        13
15         4
16        11
17        22
18         0
19         2
20         7
21         9
22         2
23        10
24        10
25        12
26        12
27        14
28        15
29         8
          ..
867843    17
867844    22
867845    18
867846    17
867847    16
867848    19
867849    16
867850     9
867851    22
867852    18
867853    11
867854    12
867855    10
867856    10
867857     8
867858    18
867859     7
867860     0
867861     8
867862    14
867863    15
867864    19
867865    14
867866    12
867867    10
867868    17
867869    22
867870    22
867871     5
867872    12
Name: Hour, Length: 867873, dtype: int64

In [60]:
dataset['HourofDay']

0               MORNING
1         EARLY_Morning
2                 Night
3               MORNING
4               MORNING
5               MORNING
6               MORNING
7               MORNING
8               MORNING
9               MORNING
10              MORNING
11        EARLY_Morning
12              MORNING
13              MORNING
14              MORNING
15        EARLY_Morning
16              MORNING
17              MORNING
18                Night
19        EARLY_Morning
20              MORNING
21              MORNING
22        EARLY_Morning
23              MORNING
24              MORNING
25              MORNING
26              MORNING
27              MORNING
28              MORNING
29              MORNING
              ...      
867843          MORNING
867844          MORNING
867845          MORNING
867846          MORNING
867847          MORNING
867848          MORNING
867849          MORNING
867850          MORNING
867851          MORNING
867852          MORNING
867853          